# Rec Sys of Answerers for StackOverflow
## 1 - Collaborative and Content Based Approach

Loading Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import norm
from sklearn.model_selection import train_test_split
import ast
from tqdm import tqdm
from functools import lru_cache

Loading Data

In [2]:
df_question = pd.read_csv("dataset/questions_2019.csv")
df_answer = pd.read_csv("dataset/answers_2019.csv")

In [3]:
df_question.head()

,QuestionId,QuestionOwnerId,QuestionTitle,QuestionTags,QuestionVotes,QuestionCreationDate,AnswerCount
0,54936100,3419772,R 3.5.2: Error in loading stock data from zoo,['r'],0,2019-03-01 00:00:10,1
1,54936106,3997132,Different behaviour of range with lodash/fp,"['functional-programming', 'lodash']",1,2019-03-01 00:01:48,1
2,54936108,4992551,webPack dev server proxy rewrite URLs in response,"['webpack', 'webpack-dev-server']",2,2019-03-01 00:01:55,1
3,54936109,2239552,EF Core 2.0 Global Filter,['c#'],0,2019-03-01 00:02:36,0
4,54936112,5505171,Clustered column chart in C# using Chart in Wi...,"['c#', 'winforms', 'charts', 'column-chart']",0,2019-03-01 00:02:47,0


In [4]:
df_answer.head()

,QuestionId,AnswerOwnerId,AnswerVotes,AnswerCreationDate
0,56140111,10245958,3,2019-05-15 00:07:23
1,56140157,1226963,0,2019-05-15 00:12:34
2,56140125,6841773,0,2019-05-15 00:12:44
3,56140150,1440565,5,2019-05-15 00:12:52
4,56140150,11015427,15,2019-05-15 00:16:46


Transfoming list of tags

In [5]:
df_question['QuestionTags'] = df_question['QuestionTags'].apply(ast.literal_eval)

In [6]:
df_question

,QuestionId,QuestionOwnerId,QuestionTitle,QuestionTags,QuestionVotes,QuestionCreationDate,AnswerCount
0,54936100,3419772,R 3.5.2: Error in loading stock data from zoo,[r],0,2019-03-01 00:00:10,1
1,54936106,3997132,Different behaviour of range with lodash/fp,"[functional-programming, lodash]",1,2019-03-01 00:01:48,1
2,54936108,4992551,webPack dev server proxy rewrite URLs in response,"[webpack, webpack-dev-server]",2,2019-03-01 00:01:55,1
3,54936109,2239552,EF Core 2.0 Global Filter,[c#],0,2019-03-01 00:02:36,0
4,54936112,5505171,Clustered column chart in C# using Chart in Wi...,"[c#, winforms, charts, column-chart]",0,2019-03-01 00:02:47,0
...,...,...,...,...,...,...,...
878713,54272409,10245420,Selenium iframe Data issue - python,"[python, selenium, selenium-webdriver, iframe]",1,2019-01-19 23:57:14,1
878714,54272411,8906835,Problem on adding Bootstrap carousel component,"[bootstrap-4, bootstrap-carousel]",0,2019-01-19 23:57:26,1
878715,54272412,10893334,NativeScript Vue nativescript-sqlite cannot as...,"[sqlite, nativescript, nativescript-vue]",0,2019-01-19 23:57:36,1
878716,54272414,10564619,How to fix 'TypeError' that comes up for some ...,"[python, typeerror]",1,2019-01-19 23:57:54,1


### Filtering Dataset to include only top 5 tags

In [7]:
tags_to_filter = {'java', 'python', 'javascript', 'c#', 'android'}  # Faster lookup structure


# Filter rows where any tag matches one of the target tags
df_question = df_question[df_question['QuestionTags'].apply(lambda tags: any(tag in tags_to_filter for tag in tags))]


## Prototype 1 - Approach Collaborative Filtering


- **Goal:** Find answerers to new posted question.
  
- **Approach:** Collaborative Filtering (Item Based) --> Utilize question tags of previous answered questions, to indicate the most appropriate user.

- **Steps:**

1. Create Matrix of User-Tags
2. Calculate cosine similarity between tags
3. Calculate the the score of the user given the number of times they answerered certain question and the similarity of that tag with the the tags of the new question.

### Dividing the question dataset into Train and Test

In [8]:
df_question

,QuestionId,QuestionOwnerId,QuestionTitle,QuestionTags,QuestionVotes,QuestionCreationDate,AnswerCount
3,54936109,2239552,EF Core 2.0 Global Filter,[c#],0,2019-03-01 00:02:36,0
4,54936112,5505171,Clustered column chart in C# using Chart in Wi...,"[c#, winforms, charts, column-chart]",0,2019-03-01 00:02:47,0
6,54936118,11072710,Getting location of tweets in Twitter API and ...,"[python, twitter, tweepy]",0,2019-03-01 00:03:33,1
8,54936122,9695341,Android version no longer working after upgrad...,"[android, gradle, dart, flutter]",1,2019-03-01 00:04:10,1
9,54936125,8821525,Plotly gives an empty field as output in jupyt...,"[python, jupyter-notebook, plotly, jupyter, ju...",27,2019-03-01 00:04:50,4
...,...,...,...,...,...,...,...
878707,54272387,6865540,Google API + Service Account for impersonate user,"[python, python-3.x, google-api, google-worksp...",2,2019-01-19 23:54:02,0
878711,54272397,10764633,Grouping not working as expected returning mor...,[c#],0,2019-01-19 23:55:52,1
878712,54272404,5615842,How to remove style element from page via Java...,"[javascript, python, selenium]",0,2019-01-19 23:56:36,0
878713,54272409,10245420,Selenium iframe Data issue - python,"[python, selenium, selenium-webdriver, iframe]",1,2019-01-19 23:57:14,1


In [9]:
# Ordering dataframe before split
# Convert 'QuestionCreationDate' to datetime format using .loc
df_question.loc[:, 'QuestionCreationDate'] = pd.to_datetime(df_question['QuestionCreationDate'])

# Sort the DataFrame by 'QuestionCreationDate' without using inplace
df_question = df_question.sort_values(by='QuestionCreationDate')

In [10]:
# Determine the split index
split_index = int(len(df_question) * 0.9) #10% for test 

# Split the DataFrame into training and testing sets
train_data = df_question.iloc[:split_index]
test_data = df_question.iloc[split_index:]

In [11]:
test_data

,QuestionId,QuestionOwnerId,QuestionTitle,QuestionTags,QuestionVotes,QuestionCreationDate,AnswerCount
569003,56549556,9375007,JS Setting background position to a random num...,"[javascript, html]",0,2019-06-11 18:14:13,1
569011,56549579,11525551,C# Windows Forms - Messagebox showing once mor...,"[c#, message, messagebox, box]",0,2019-06-11 18:15:22,2
569015,56549597,11192937,Mesh manipulation very slow,"[c#, unity-game-engine, optimization]",4,2019-06-11 18:16:25,2
569019,56549609,11632718,Python: multiple simultaneous requests,"[python, multithreading, python-requests]",0,2019-06-11 18:17:45,1
569020,56549610,4589310,SQL query to order by a special condition,"[python, sql, postgresql, sqlalchemy]",0,2019-06-11 18:17:50,1
...,...,...,...,...,...,...,...
156647,56828742,8928024,What are the value states of read/write in sel...,"[python, python-3.x, client-server, bitwise-op...",2,2019-06-30 23:36:28,1
156648,56828743,9513001,Python seaborn / matplotlib - show frequency i...,"[python, matplotlib, seaborn]",0,2019-06-30 23:36:37,1
156650,56828747,11721483,Is it valid to include declarations and ternar...,"[javascript, function, functional-programming,...",-1,2019-06-30 23:37:49,1
156658,56828780,11661267,Dataframe how to update a column based many st...,"[python, pandas, numpy, dataframe]",0,2019-06-30 23:45:16,2


Merging QuestionTags to UserAnswerId

In [12]:
df_merged = train_data[["QuestionId","QuestionTags"]].merge(df_answer[["QuestionId", "AnswerOwnerId"]], how='inner', on = "QuestionId", )

Getting all tags of a user -> for checking purposes

In [13]:
# df_grouped_tags = (
#     df_merged.groupby("AnswerOwnerId")["QuestionTags"]
#     .apply(lambda tags: list(set(tag for sublist in tags for tag in sublist)))  # Flatten and deduplicate tags
#     .reset_index()
# )

# df_grouped_tags.columns = ["AnswerOwnerId", "CombinedTags"]

### Indexing tags and users

In [14]:
dic_merged = df_merged.to_dict(orient="records")

In [15]:
unique_tags = list({tag for question in dic_merged for tag in question['QuestionTags']})
unique_users = list({question['AnswerOwnerId'] for question in dic_merged})

tag_to_index = {tag: idx for idx, tag in enumerate(unique_tags)}
user_to_index = {user: idx for idx, user in enumerate(unique_users)}

### Creating User-Tag Matrix

In [16]:
# Step 2: Create a Sparse User-Tag Matrix
rows, cols, data = [], [], []
for question in dic_merged:
    user_idx = user_to_index[question['AnswerOwnerId']]
    for tag in question['QuestionTags']:
        tag_idx = tag_to_index[tag]
        rows.append(user_idx)
        cols.append(tag_idx)
        data.append(1)

# Create a sparse matrix 
user_tag_matrix_sparse = csr_matrix((data, (rows, cols)), shape=(len(unique_users), len(unique_tags)))


### Calculating Tags Similarity Matrix

In [17]:
# Compute Tag Similarity
cos_sim_matrix = cosine_similarity(user_tag_matrix_sparse.T)

### Recommending User

In [18]:
test_data

,QuestionId,QuestionOwnerId,QuestionTitle,QuestionTags,QuestionVotes,QuestionCreationDate,AnswerCount
569003,56549556,9375007,JS Setting background position to a random num...,"[javascript, html]",0,2019-06-11 18:14:13,1
569011,56549579,11525551,C# Windows Forms - Messagebox showing once mor...,"[c#, message, messagebox, box]",0,2019-06-11 18:15:22,2
569015,56549597,11192937,Mesh manipulation very slow,"[c#, unity-game-engine, optimization]",4,2019-06-11 18:16:25,2
569019,56549609,11632718,Python: multiple simultaneous requests,"[python, multithreading, python-requests]",0,2019-06-11 18:17:45,1
569020,56549610,4589310,SQL query to order by a special condition,"[python, sql, postgresql, sqlalchemy]",0,2019-06-11 18:17:50,1
...,...,...,...,...,...,...,...
156647,56828742,8928024,What are the value states of read/write in sel...,"[python, python-3.x, client-server, bitwise-op...",2,2019-06-30 23:36:28,1
156648,56828743,9513001,Python seaborn / matplotlib - show frequency i...,"[python, matplotlib, seaborn]",0,2019-06-30 23:36:37,1
156650,56828747,11721483,Is it valid to include declarations and ternar...,"[javascript, function, functional-programming,...",-1,2019-06-30 23:37:49,1
156658,56828780,11661267,Dataframe how to update a column based many st...,"[python, pandas, numpy, dataframe]",0,2019-06-30 23:45:16,2


In [19]:
## Example to 1 question
# tags = test_data["QuestionTags"].iloc[0]
# tags

tags = ["r", "radar-chart", "spider-chart"]

In [20]:
def recommend_users_collaborative(tags):
    # Get indices of the tags
    tag_indices = [tag_to_index[tag] for tag in tags if tag in tag_to_index]
    if len(tag_indices)==0:
        return [] ## No tag found to make the suggestion
    
    # Combine similarities for the requested tags
    similarity_tags = [cos_sim_matrix[:, tag_index] for tag_index in tag_indices]
    combined_similarity = sum(similarity_tags)

    # Compute user scores 
    user_scores_vector = user_tag_matrix_sparse.dot(combined_similarity)

    # Map scores to users
    user_scores = {unique_users[idx]: score for idx, score in enumerate(user_scores_vector)}

    # Rank users by scores
    ranked_users = sorted(user_scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_users

In [21]:
return_1 = recommend_users_collaborative(tags)

## Prototype 2 - Approach Content Based-Filtering


- **Goal:** Find answerers to new posted question.
  
- **Approach:** Content Based-Filtering --> Utilize question tags of previous answered questions, to indicate the most appropriate user.

- **Steps:**

1. Create Matrix of User-Tags
2. Create vector of to the new tag based on the Matrix of User-tags 
3. Calculate cosine similarity for each user with vector created in step 2
4. Order list to show the most recommended users

In [22]:
# We are going start using some User-Tag matrix of the previous approach

def recommend_users_content(tags):
    # Get index of the tags
    tag_indices = [tag_to_index[tag] for tag in tags if tag in tag_to_index]
    if len(tag_indices)==0:
        return [] ## No tag found to make the suggestion
            
    # Create vector of the tags in question
    question_vector = np.zeros(user_tag_matrix_sparse.shape[1])
    question_vector[tag_indices] = 1
    question_vector= question_vector.reshape(1, -1) #Reshaping array to calculate similarity

    user_scores_vector = cosine_similarity(user_tag_matrix_sparse, question_vector)
    user_scores_vector = user_scores_vector.flatten()

    # Map scores to users
    user_scores = {unique_users[idx]: score for idx, score in enumerate(user_scores_vector)}

    # Rank users by scores
    ranked_users = sorted(user_scores.items(), key=lambda x: x[1], reverse=True)
    # return ranked_users[:10]
    return ranked_users


In [23]:
return_2 = recommend_users_content(tags)

## Evaluation

Top-N Accuracy

In [24]:
df_merged_test = test_data[["QuestionId","QuestionTags"]].merge(df_answer[["QuestionId", "AnswerOwnerId"]], how='inner', on = "QuestionId", )

Joining the Users to form a single ground truth

In [25]:
# Convert QuestionTags from lists to tuples
df_merged_test["QuestionTags"] = df_merged_test["QuestionTags"].apply(tuple)

# Group AnswerOwnerId by QuestionId and QuestionTags
grouped_df = (
    df_merged_test.groupby(["QuestionId", "QuestionTags"])["AnswerOwnerId"]
    .apply(list)  # Combine AnswerOwnerId values into a list
    .reset_index()
    .rename(columns={"AnswerOwnerId": "GroundTruth"})
)

In [26]:
ground_truth_dic = grouped_df.to_dict(orient='records')

Calculate the results for all questions

In [27]:
def top_n_accuracy(users_ground_truth_list, users_recommendation_list, top_n):
    tp = 0
    
    for users_ground_truth, users_rec in zip(users_ground_truth_list, users_recommendation_list):    
        # Checking if it is in the answer
        users_recommendation = users_rec[:top_n]
        if set(users_recommendation) & set(users_ground_truth):
            tp += 1
            
    print(tp)
    return tp/len(users_ground_truth_list)

In [28]:
users_ground_truth = [ground_truth_item["GroundTruth"] for ground_truth_item in ground_truth_dic]
tags_ground_truth = [tuple(sorted(ground_truth_item["QuestionTags"])) for ground_truth_item in ground_truth_dic]

Collaborative

In [29]:
top_n = [5, 10, 20, 50]

users_recommendation_list = []
for tags_tuple in tqdm(tags_ground_truth, desc="Processing Recommendations"):
    recommendation = recommend_users_collaborative(tags_tuple)[:max(top_n)]
    users_rec = [rec[0] for rec in recommendation]
    users_recommendation_list.append(users_rec)

Processing Recommendations: 100%|██████████| 28052/28052 [33:21<00:00, 14.02it/s]


In [30]:
# Evaluate Recommendation
results = []

for top in top_n:
    result = {
        "acc": top_n_accuracy(users_ground_truth, users_recommendation_list, top),
        "top": top,
        "type": "collaborative"
    }
    
    print(result)
    results.append(result)

1718
{'acc': 0.06124340510480536, 'top': 5, 'type': 'collaborative'}
2382
{'acc': 0.08491373164123771, 'top': 10, 'type': 'collaborative'}
3367
{'acc': 0.12002709254242122, 'top': 20, 'type': 'collaborative'}
5064
{'acc': 0.18052188792242976, 'top': 50, 'type': 'collaborative'}


Content

In [31]:
top_n = [5, 10, 20, 50]

users_recommendation_list_content = []
for tags_tuple in tqdm(tags_ground_truth, desc="Processing Recommendations"):
    recommendation = recommend_users_content(tags_tuple)[:max(top_n)]
    users_rec = [rec[0] for rec in recommendation]
    users_recommendation_list_content.append(users_rec)

Processing Recommendations: 100%|██████████| 28052/28052 [26:53<00:00, 17.39it/s]


In [ ]:
# Evaluate Recommendation
for top in top_n:
    result = {
        "acc": top_n_accuracy(users_ground_truth, users_recommendation_list_content, top),
        "top": top,
        "type": "content"
    }
    
    print(result)
    results.append(result)

634
{'acc': 0.022600884072436902, 'top': 5, 'type': 'content'}
1057
{'acc': 0.03768002281477256, 'top': 10, 'type': 'content'}
1598
{'acc': 0.056965635248823615, 'top': 20, 'type': 'content'}
2524
{'acc': 0.08997575930414943, 'top': 50, 'type': 'content'}


Random Pick

In [33]:
import random
users = list(set(df_merged["AnswerOwnerId"].to_list()))
sample = [random.sample(users, max(top_n)) for i in range(len(users_ground_truth))]

for top in top_n:
    result = {
        "acc": top_n_accuracy(users_ground_truth, sample, top),
        "top": top,
        "type": "random"
    }
    print(result)
    results.append(result)

0
{'acc': 0.0, 'top': 5, 'type': 'random'}
2
{'acc': 7.129616426636246e-05, 'top': 10, 'type': 'random'}
3
{'acc': 0.00010694424639954371, 'top': 20, 'type': 'random'}
12
{'acc': 0.00042777698559817484, 'top': 50, 'type': 'random'}


Saving Results

In [34]:
pd.DataFrame(results).to_csv("setup1.csv", index=False)